In [3]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("speech.txt")

d:\GenAI\LangChain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
text_docs = loader.load()
texts = text_docs[0:3]

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")

In [6]:
# web based loader
%pip install langchain-community bs4
from langchain_community.document_loaders import WebBaseLoader
import bs4

# load, chunk and index the content of the html page
# bs4 -> It’s a web scraping library used to extract clean text from HTML, which LangChain then uses as data.
loader = WebBaseLoader(web_path=("https://en.wikipedia.org/wiki/Generative_pre-trained_transformer",),
                       bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                           class_=("mw-parser-output", "mw-content-ltr mw-parser-output", "mw-references-wrap mw-references-columns",)
                       )),)


Note: you may need to restart the kernel to use updated packages.


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
text_documents=loader.load()

In [8]:
# pdf reader
%pip install pypdf
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('NIPS-2017-attention-is-all-you-need-Paper.pdf')
docs=loader.load()

Note: you may need to restart the kernel to use updated packages.


TILL HERE THE ##LOAD DATASET## PART HS BEEN DONE
LETS MOVE TO TANSFORM PART

In [9]:
# we first converted the documents into smaller chunks

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

documents = text_splitter.split_documents(docs)

In [ ]:
# BELOW CODE USES GEMINI EMBEDDINGS WHICH ARE PAID. SO WE HAVE USED(IN THE FOLLOWING CELL) LOCAL EMBEDDINGS(HUGGINGFACE)


# vector embeddings and vector store
# first we will reate the vector embeddings of every chunked document and then store them in a vector database(store) for efficient retrieval.
# %pip install langchain
%pip install langchain-google-genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

db = Chroma.from_documents(
    documents[:10],
    embeddings,
    collection_name="attention_is_all_you_need"
)

Note: you may need to restart the kernel to use updated packages.


GoogleGenerativeAIError: Error embedding content (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0, model: embedding-001\n* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0, model: embedding-001\n* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0, model: embedding-001\n* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0, model: embedding-001\nPlease retry in 21.311928531s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleap

In [10]:
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install sentence-transformers
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

db = Chroma.from_documents(
    documents[:10],
    embeddings,
    collection_name="attention_is_all_you_need"
)


Note: you may need to restart the kernel to use updated packages.


C:\Users\DELL\AppData\Local\Temp\ipykernel_21820\3463643645.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [18]:
%pip install faiss-cpu

   ---------------------------------------- 0.0/18.9 MB ? eta -:--:--
   --- ------------------------------------ 1.8/18.9 MB 11.2 MB/s eta 0:00:02
   --------- ------------------------------ 4.5/18.9 MB 11.2 MB/s eta 0:00:02
   -------------- ------------------------- 6.8/18.9 MB 11.7 MB/s eta 0:00:02
   ------------------- -------------------- 9.4/18.9 MB 11.3 MB/s eta 0:00:01
   ------------------------ --------------- 11.8/18.9 MB 11.2 MB/s eta 0:00:01
   ----------------------------- ---------- 13.9/18.9 MB 11.0 MB/s eta 0:00:01
   ---------------------------------- ----- 16.3/18.9 MB 11.1 MB/s eta 0:00:01
   ------------------------------------- -- 17.8/18.9 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------  18.9/18.9 MB 10.3 MB/s eta 0:00:01
   ---------------------------------------  18.9/18.9 MB 10.3 MB/s eta 0:00:01
   ---------------------------------------- 18.9/18.9 MB 7.6 MB/s  0:00:02
Note: you may need to restart the kernel to use updated packages.


'The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU\n[20], ByteNet [15] and ConvS2S [8], all of which use convolutional neural networks as basic building\nblock, computing hidden representations in parallel for all input and output positions. In these models,\nthe number of operations required to relate signals from two arbitrary input or output positions grows\nin the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes\nit more difﬁcult to learn dependencies between distant positions [ 11]. In the Transformer this is\nreduced to a constant number of operations, albeit at the cost of reduced effective resolution due\nto averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as\ndescribed in section 3.2.\nSelf-attention, sometimes called intra-attention is an attention mechanism relating different positions'

In [12]:
# FAISS vector DB
from langchain_community.vectorstores import FAISS
faiss_db = FAISS.from_documents(
    documents[:10],
    embeddings
)

In [13]:
faiss_db


In [15]:
query = "What is attention mechanism in transformers?"
result=faiss_db.similarity_search(query)
result[0].page_content

'The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU\n[20], ByteNet [15] and ConvS2S [8], all of which use convolutional neural networks as basic building\nblock, computing hidden representations in parallel for all input and output positions. In these models,\nthe number of operations required to relate signals from two arbitrary input or output positions grows\nin the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes\nit more difﬁcult to learn dependencies between distant positions [ 11]. In the Transformer this is\nreduced to a constant number of operations, albeit at the cost of reduced effective resolution due\nto averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as\ndescribed in section 3.2.\nSelf-attention, sometimes called intra-attention is an attention mechanism relating different positions'

In [16]:
query = "What is attention mechanism in transformers?"
result=db.similarity_search(query)
result[0].page_content

'The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU\n[20], ByteNet [15] and ConvS2S [8], all of which use convolutional neural networks as basic building\nblock, computing hidden representations in parallel for all input and output positions. In these models,\nthe number of operations required to relate signals from two arbitrary input or output positions grows\nin the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes\nit more difﬁcult to learn dependencies between distant positions [ 11]. In the Transformer this is\nreduced to a constant number of operations, albeit at the cost of reduced effective resolution due\nto averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as\ndescribed in section 3.2.\nSelf-attention, sometimes called intra-attention is an attention mechanism relating different positions'

In [42]:
# design chatbot with retrieval
# from langchain_community.chat_models import GoogleGenerativeAIChat
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7,
)

from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template("""
                                        You are a helpful AI assistant. Use the following context to answer the question.
                                        <context>
                                        {context}
                                        </context>
                                        Question: {input}
                                        """)

In [43]:
# from langchain import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

# Chain introduction
# create stuff document chain
# from langchain.chains.combine_documents import create_stuff_documents_chain

# from chains import combine_documents


stuff_chain = create_stuff_documents_chain(
    llm=chat_model,
    prompt=prompt
) 

In [44]:
retriever = faiss_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000016BF7D9A890>, search_kwargs={})

In [45]:
from langchain_classic.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(
    # llm=chat_model,
    retriever,
    stuff_chain
)

In [49]:
response=retriever_chain.invoke({"input": "The encoder is composed of a stack of how many identical layers.?"})

In [50]:
response['answer']

'The encoder is composed of a stack of **N = 6** identical layers.'